In [3]:
import pandas as pd
from geopy.geocoders import Nominatim
#Importación del módulo csv que necesitaremos para leer y escribir en nuestros csv
import csv
#Importación del módulo shutil que usaremos para crear una copia de nuestro csv original
import shutil
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
import pyrematch as re
from shapely.geometry import shape
import numpy as np
import networkx as nx
from time import time 
from pyproj import CRS
from collections import OrderedDict
import operator
from functools import reduce


# Insertar costos a los arcos de Santiago

**Creamos el grafo de Santiago**

In [4]:
place_name = "Provincia de Santiago, Región Metropolitana de Santiago, Chile"
graph = ox.graph_from_place(place_name, network_type = "drive", simplify = False)

In [5]:
ox.speed.add_edge_speeds(graph, hwy_speeds=None, fallback=None, precision=1)

**Obtenemos los df de nodos y arcos**

In [6]:
nodes, edges = ox.graph_to_gdfs(graph)

In [7]:
contador = 0

while contador < 6:
    for n in edges.index:
        if contador < 6:
            print(edges['u'][n])
        contador += 1
        
edges[:10]

386131
386132
386138
386138
386139
386139


,osmid,oneway,lanes,name,highway,length,speed_kph,access,maxspeed,bridge,tunnel,ref,junction,width,service,landuse,geometry,u,v,key
0,121815695,True,4,Avenida Libertador Bernardo O'Higgins,primary,53.246,52.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-70.64237 -33.44155, -70.64192 -33...",386131,252521298,0
1,121815695,True,4,Avenida Libertador Bernardo O'Higgins,primary,20.476,52.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-70.64325 -33.44205, -70.64305 -33...",386132,252521312,0
2,121815664,True,NaN,Enrique Mac Iver,primary,16.056,52.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-70.64625 -33.44290, -70.64628 -33...",386138,14805063,0
3,121815695,True,4,Avenida Libertador Bernardo O'Higgins,primary,45.325,52.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-70.64625 -33.44290, -70.64578 -33...",386138,500862146,0
4,23321898,True,2,San Francisco,secondary,16.502,49.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-70.64729 -33.44314, -70.64715 -33...",386139,6629836903,0
5,926197834,True,4,Avenida Libertador Bernardo O'Higgins,primary,100.015,52.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-70.64729 -33.44314, -70.64625 -33...",386139,386138,0
6,407593363,True,3,Avenida Libertador Bernardo O'Higgins,primary,61.410,52.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-70.64764 -33.44307, -70.64828 -33...",386140,4439790028,0
7,407593364,True,NaN,San Antonio,secondary,15.720,49.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-70.64764 -33.44307, -70.64761 -33...",386140,500862085,0
8,407593363,True,3,Avenida Libertador Bernardo O'Higgins,primary,8.648,52.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-70.64898 -33.44339, -70.64907 -33...",386145,7569146928,0
9,448518898,False,NaN,NaN,primary_link,16.654,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-70.64898 -33.44339, -70.64896 -33...",386145,646864662,0


**Agregamos columna de tiempo de viaje**

In [8]:
edges['Tiempos de Viaje'] = 0
edges['Tiempos de Viaje'] = edges.apply(lambda x: ((edges['length']/1000)/edges['speed_kph'])) #horas

**Creamos función que disminuirá la velocidad  del los arcos, dependiendo de la hora y minuto del día**

**Funciones que servirán para dar efecto congestión**

In [9]:
def interpolacion_lineal(x,x1,x2,y1,y2):
    y = ((y2-y1)/(x2-x1))*(x-x1)+y1
    return(y)

In [10]:
def interpolacion_cuadratica(x, fx):
    b0 = fx[0]
    b1 = (fx[1]-fx[0])/(x[1]-x[0])
    b2 = (((fx[2]-fx[1])/(x[2]-x[1]))-((fx[1]-fx[0])/(x[1]-x[0])))/(x[2]-x[0])
    f2 = b0+b1*(x[3]-x[0])+b2*(x[3]-x[0])*(x[3]-x[1])
    return(f2)

**Función tiempo final aplicada a la red**

In [11]:
def actualizar_tiempo(x, y, mes, hora, minutos): #x es velocidad e y es distancia
    y = y/1000
    dic_dism = {1:[(20.19/43.15),((20.19+14.09)/43.15),(18.51/43.15)],2:[(21.03/43.15),((21.03+14.09)/43.15),(19.14/43.15)], 3:[(16.89/43.15),((16.89+14.09)/43.15),(17.39/43.15)], 4:[(16.93/43.15),((16.93+14.09)/43.15),(17.38/43.15)], 5:[(17.19/43.15),((17.19+14.09)/43.15),(17.17/43.15)], 6:[(17.34/43.15),((17.34+14.09)/43.15),(17.08/43.15)], 7:[(18.46/43.15),((18.46+14.09)/43.15),(17.21/43.15)], 8:[(16.76/43.15),((16.76+14.09)/43.15),(16.94/43.15)], 9:[(17.15/43.15),((17.15+14.09)/43.15),(17.24/43.15)], 10:[(16.78/43.15),((16.78+14.09)/43.15),(17.02/43.15)], 11:[(16.57/43.15),((16.57+14.09)/43.15),(17.03/43.15)], 12:[(18.11/43.15),((18.11+14.09)/43.15),(17.13/43.15)]}
    #Punta mañana [0], Valle [1], Punta Tarde [2]
    if ((7 <= hora <= 8) and (0 <= minutos <= 59)):
        x = interpolacion_cuadratica([5, 8, 10.5, (hora+(minutos/60))], [x, x*dic_dism[mes][0], interpolacion_lineal(10.5,8,13.5,x*dic_dism[mes][0],x*dic_dism[mes][1])])
    elif ((9 <= hora <= 17) and (0 <= minutos <= 59)):
        x = interpolacion_cuadratica([10.5, 13.5, 16.25,(hora +(minutos/60))],[interpolacion_lineal(10.5,8,13.5,x*dic_dism[mes][0],x*dic_dism[mes][1]),x*dic_dism[mes][1], interpolacion_lineal(16.25,13.5,19,x*dic_dism[mes][1],x*dic_dism[mes][2])])
    elif ((18 <= hora <= 19) and (0 <= minutos <= 59)):
        x = interpolacion_cuadratica([16.25, 19, 22, (hora+(minutos/60))],[interpolacion_lineal(16.25,13.5,19,x*dic_dism[mes][1],x*dic_dism[mes][2]),x*dic_dism[mes][2],x])
    return (y/x) #horas

**Ahora probamos la función para los diferentes horarios**

00:03 hrs.

In [12]:
#nodes, edges = ox.graph_to_gdfs(graph)

In [13]:
#actualizar_tiempo(x, y, mes, hora, minutos)
#edges['Tiempo de Viaje'][10:20]

8:33 hrs.

In [14]:
#nodes, edges = ox.graph_to_gdfs(graph)

In [15]:
#actualizar_tiempo(x, y, mes, hora, minutos)
#edges['Tiempo de Viaje'][10:20]

10:32 hrs.

In [16]:
#nodes, edges = ox.graph_to_gdfs(graph)

In [17]:
#actualizar_tiempo(x, y, mes, hora, minutos)
#edges['Tiempo de Viaje'][10:20]

13:00 hrs.

In [18]:
#nodes, edges = ox.graph_to_gdfs(graph)

In [19]:
#actualizar_tiempo(x, y, mes, hora, minutos)
#edges['Tiempo de Viaje'][10:20]

13:30 hrs.

In [20]:
#nodes, edges = ox.graph_to_gdfs(graph)

In [21]:
#actualizar_tiempo(x, y, mes, hora, minutos)
#edges['Tiempo de Viaje'][10:20]

18:20 hrs.

In [22]:
#nodes, edges = ox.graph_to_gdfs(graph)

In [23]:
#actualizar_tiempo(x, y, mes, hora, minutos)
#edges['Tiempo de Viaje'][10:20]

**Función Dijsktra dependiente del tiempo**

In [24]:
def dijkstra_tiempo(graph, src, dest, mes, hora, minutos, visited=[], times={}, predecessors={}):
    
    """ calculates a shortest path tree routed in src
    """    
    #print('ENTRO A LA FUNCION------------------')
    if src == dest:
        #print('LLEGÓ AL DESTINO------------------')
        # We build the shortest path and display it
        path=[]
        pred=dest
        while pred != None:
            path.append(pred)
            pred=predecessors.get(pred,None)
        # reverses the array, to display the path nicely
        readable=path[0]
        #for index in range(1,len(path)): 
         #   readable = path[index],'--->',readable
        #prints it 
        #print("path: ",readable,",   cost=", str(times[dest]))
        return(path, times[dest])
        
    else : 
        #print('AUN NO LLEGA AL DESTINO------------------')
        # if it is the initial  run, initializes the cost
        if not visited: 
            times[src] = 0
        # visit the neighbors
        #Actualizar tiempos
        for neighbor in graph[src] :
            if neighbor not in visited:
                #print('VECINO VISITADO------------------', neighbor)
                x = graph[src][neighbor][0]['speed_kph']
                y = graph[src][neighbor][0]['length']
                act_time = actualizar_tiempo(x, y, mes, hora, minutos) #horas
                new_time = times[src] + act_time # tiempo acumulado en horas
                if new_time < times.get(neighbor,float('inf')):
                    times[neighbor] = new_time
                    predecessors[neighbor] = src
        #print('YA VISITÓ A TODOS LOS VECINOS DE------------------', src)
        # mark as visited
        visited.append(src)
        # now that all neighbors have been visited: recurse                         
        # select the non visited node with lowest distance 'x'
        # run Dijskstra with src='x'
        unvisited={}
        for k in graph:
            if k not in visited:
                unvisited[k] = times.get(k,float('inf'))        
        nodo_actual = min(unvisited, key=unvisited.get) ## AQUI OCUPAR HEAP
        #print('CAMBIÓ DE NODO, AHORA ESTÁ EN------------------', nodo_actual)
        return dijkstra_tiempo(graph, nodo_actual, dest, mes, hora, minutos, visited, times, predecessors)        

In [25]:
from collections import defaultdict
from heapq import *

def dijkstra(edges, orig, dest,  mes, hora, minutos): # f: inicio | t: destino
    g = defaultdict(list) # diccionario de lista?
    
    for n in edges.index:
        x = edges['speed_kph'][n]
        y = edges['length'][n]
        act_time = actualizar_tiempo(x, y, mes, hora, minutos)
        edges['Tiempos de Viaje'][n] = act_time
        g[edges['u'][n]].append((act_time, edges['v'][n])) # para cada par de nodos, agregar al dicc la tupla de (nodo alcanzado, dist)
    
    q, seen, mins = [(0,orig,())], set(), {orig: 0} # q: no vistos | seen: vistos | mins: mínimos?
    while q:
        (cost,v1,path) = heappop(q) # mientras q no esté vacío
        if v1 not in seen: # si v1 no está en vistos
            seen.add(v1) # añador a los vistos
            path = (v1, path) 
            if v1 == dest: 
                return (cost, path) # si es el destino
                

            for c, v2 in g.get(v1, ()): # obtengo los nodos alcanzados por v1 (costo, nodo_alc)
                if v2 in seen: continue # si v2 está en vistos
                prev = mins.get(v2, None) # obteneer el mínimo de ese nodo
                next = cost + c # le suma el costo?
                if prev is None or next < prev: # si no hay precedor o next es menor al prev
                    mins[v2] = next # mínimo es next
                    heappush(q, (next, v2, path)) # se agrega a q

    return float("inf"), None

In [26]:
dijkstra(edges[:10], 386131, 252521298, 6, 7, 30)

C:\Users\Berenice\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Berenice\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


(0.002290768322909714, (252521298, (386131, ())))

In [27]:
from collections import defaultdict
from heapq import *

def dijkstra(edges, f, t): # f: inicio | t: destino
    g = defaultdict(list) # diccionario de lista?
    
    for n in edges.index:
        g[edges['u'][n]].append((edges['Tiempos de Viaje'][n], edges['v'][n])) # para cada par de nodos, agregar al dicc la tupla de (nodo alcanzado, dist)

    q, seen, mins = [(0,f,())], set(), {f: 0} # q: no vistos | seen: vistos | mins: mínimos?
    while q:
        
        (cost,v1,path) = heappop(q) # mientras q no esté vacío
        if v1 not in seen: # si v1 no está en vistos
            seen.add(v1) # añador a los vistos
            path = (v1, path) 
            
            if v1 == t: 
                return (cost, path) # si es el destino
                
            for c, v2 in g.get(v1, ()): # obtengo los nodos alcanzados por v1 (costo, nodo_alc)
                if v2 in seen: continue # si v2 está en vistos
                prev = mins.get(v2, None) # obteneer el mínimo de ese nodo
                next = cost + c # le suma el costo
                if prev is None or next < prev: # si no hay precedor o next es menor al prev
                    mins[v2] = next # mínimo es next
                    heappush(q, (next, v2, path)) # se agrega a q

    return float("inf"), None 

In [28]:
dijkstra(edges[:10], 386131, 252521298)

(0.002290768322909714, (252521298, (386131, ())))

In [29]:
def subgrafo(graph, src, dest, k):
    k_rutas = ox.distance.k_shortest_paths(graph, src, dest, k, weight='Tiempos de Viaje')
    lista_sets = []
    for counter, path in enumerate(k_rutas):
        lista_sets.append(set(path))
        if counter == k-1:
             break
    nodos = list(reduce(operator.or_, lista_sets))
    subgrafo = graph.subgraph(nodos)
    return subgrafo

**Actualizamos valores de arcos en el grafo de santiago, ahora se llamará G contiene velocidad con congestión**

In [30]:
G  = ox.utils_graph.graph_from_gdfs(nodes , edges , graph_attrs = None)

In [31]:
nodesG, edgesG = ox.graph_to_gdfs(G)

**Ahora podemos calcular la ruta mínima entre dos nodos, bajo cualquier atributo de arco**

In [32]:
nodos = pd.read_csv("DFNodosCercanos.csv")

In [33]:
list_id = []
for row in nodos.iterrows():
    list_id.append(row[1][3])
    
list_id

[267454352,
 613890784,
 3536637293,
 4765300244,
 376264329,
 7326370340,
 1260237000,
 6104118724,
 6993554475,
 3485642555,
 4789079544,
 1552735798]

In [34]:
ClientesAprox = pd.read_csv("DFNodosCercanos.csv")
direccion_x =[]
direccion_y =[]
for row in ClientesAprox.iterrows():
    direccion_x.append(row[1][1])
    direccion_y.append(row[1][2])
direccion_x
direccion_y

[-70.70682140000001,
 -70.6805126,
 -70.6286314,
 -70.6097452,
 -70.6809422,
 -70.727027,
 -70.7228183,
 -70.7622699,
 -70.6991552,
 -70.76003409999998,
 -70.71078670000001,
 -70.7454208]

**Función que debe ir calculando los costos que difieren del tiempo**

(1286.7*(edges[lenght]/1000)) ---------------> Todos los arcos

(sumar + 21378) -----------------------------> Arcos salientes del depot

In [35]:
def costos_agregados(path):
    agregados = 0
    for n in range(len(path)-1):
        if path[len(path)-1] == 267454352 and agregados == 0:
            agregados += 21378
        else:
            agregados += (1286.7 * graph[path[n+1]][path[n]][0]['length'])/1000
    return (agregados)

In [36]:
# Calculate the shortest path dinamic time
def rutas_min_pares(graph, list_id, mes, hora, minutos, k):
    tiempo_inicial = time() 
    dic_rutas = {}
    G = ox.utils_graph.graph_from_gdfs(nodes , edges , graph_attrs = None)
    for n1 in list_id:
        for n2 in list_id:
            if n1 != n2 and nx.has_path(G, n1,n2):
                subg = subgrafo(graph, n1, n2, k)
                path, tiempo = dijkstra_tiempo(subg, n1, n2, mes, hora, minutos, [], {}, {})
                costo_agregado = costos_agregados(path)
                dic_rutas[(path[0], path[-1])] = [tiempo, costo_agregado]           
    tiempo_final = time() 
    tiempo_ejecucion = tiempo_final - tiempo_inicial
    print ('El tiempo de ejecucion en minutos fue:', tiempo_ejecucion/60)
    return(dic_rutas)

In [6]:
dic_rutas = (rutas_min_pares(graph, list_id, 6, 9, 0, 2))

NameError: name 'rutas_min_pares' is not defined

In [ ]:
### DIJKSTRA_tIEMPO CON HEAP

In [3]:
from heapq import *
def dijkstra_tiempo(graph, src, dest, mes, hora, minutos, visited=set(), times={}, predecessors={}):
    
    """ calculates a shortest path tree routed in src
    """    
    if src == dest:
        #print('LLEGÓ AL DESTINO------------------')
        # We build the shortest path and display it
        path=[]
        pred=dest
        while pred != None:
            path.append(pred)
            pred=predecessors.get(pred,None)
        # reverses the array, to display the path nicely
        readable=path[0]

        return(path, times[dest])
        
    else : 
        # if it is the initial  run, initializes the cost
        if not visited: 
            times[src] = 0
            
        
        for neighbor in graph[src] :
            if neighbor not in visited:
                x = graph[src][neighbor][0]['speed_kph']
                y = graph[src][neighbor][0]['length']
                act_time = actualizar_tiempo(x, y, mes, hora, minutos) #horas
                new_time = times[src] + act_time # tiempo acumulado en horas
                if new_time < times.get(neighbor,float('inf')):
                    times[neighbor] = new_time
                    predecessors[neighbor] = src

        visited.add(src)
        unvisited=[]
        
        for k in graph:
            if k not in visited:
                heappush(unvisited, (times.get(k, float('inf')), k))   

        nodo_actual = heappop(unvisited)[1] 
        return dijkstra(graph, nodo_actual, dest, mes, hora, minutos, visited, times, predecessors)

In [ ]:
dic_rutas_h = (rutas_min_pares(G, list_id, 6, 9, 0, 1))

In [ ]:
print(dic_rutas)

# -----------------------------------------------------------------------------------------------------

In [ ]:
"""# Necesitamos ahora, crear la estructura de grafo sin calles con los costos reales en los arcos entre cada par o-d
#Se importa la libreria networkx como nx

import networkx as nx

#Se importa la libreria pyplot de matplotlib como plt

import matplotlib.pyplot as plt

#Se crea un grafo asimétrico vacio

G = nx.DiGraph()

#Se crean los nodos:

#Se crean los nodos depot + clientes

G.add_nodes_from(list_id)

#Se crean los arcos entre los nodos
for a in dic_rutas:
    G.add_edge(a[0], a[1])
    G.edges[(a[0], a[1])]['Costo']= dic_rutas[a]
for n in G.nodes:
    if n == 267454352:
        G.nodes[n]['Depot'] = True
    else:
        G.nodes[n]['Depot'] = False
    
#Se dibuja el grafo

nx.draw(G, node_size = 12, font_size=1)

#Se muestra en pantalla

plt.show()"""



In [ ]:
"""G.nodes"""

In [ ]:
"""G.edges"""

In [ ]:
"""G.edges[(3485642555, 4765300244)]"""

In [ ]:
"""G.nodes[613890784]"""

# Creación de datos

In [ ]:
"""n = list_id.copy()# Clientes
camiones = [k for k in range (1,4)]
list_id.remove(267454352)
clientes = list_id
u={1: 5500, 2:5500, 3:5500}

import numpy as np
rnd=np.random
rnd.seed(0)

q={i:rnd.randint(500, 1000) for i in clientes}"""

In [ ]:
"""print(n)
print(clientes)
print(camiones)
print(u)
print(q)
print(list_id)"""

In [ ]:
"""import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
plt.scatter(direccion_x,direccion_y,color="green")

for i in clientes:
       plt.annotate('$q_{%d}=%d$'%(i,q[i]),(direccion_x[clientes.index(i)],direccion_y[clientes.index(i)]))
    
plt.plot(direccion_x[0],direccion_y[0],color='red',marker='s')
plt.annotate('DC',(direccion_x[0],direccion_y[0]))

plt.xlabel("Eje X")
plt.ylabel("Eje Y")
plt.title("Gráfico de Nodos | VRP")

plt.show()"""



In [ ]:
"dic_rutas[(3485642555, 4765300244)]"

In [ ]:
""""arcos={i for i in dic_rutas.keys()}
# esto es literal dic_rutas
#distancia={(i,j):np.hypot(direccion_x[i]-direccion_x[j],direccion_y[i]-direccion_y[j]) 
          # for i in nodos for j in nodos if i!=j}
arcos"""

In [ ]:
"""plt.figure(figsize=(12,5))
plt.scatter(direccion_x,direccion_y,color="green")

for i in clientes:
    plt.annotate('$q_{%d}$'%(i),(direccion_x[clientes.index(i)],direccion_y[clientes.index(i)]))
    
plt.plot(direccion_x[0],direccion_y[0],color='red',marker='s')
plt.annotate('DC',(direccion_x[0],direccion_y[0]))

for i, j in arcos:
    plt.plot([direccion_x[n.index(i)],direccion_x[n.index(j)]],[direccion_y[n.index(i)],direccion_y[n.index(j)]],color="blue",alpha=0.3)
    #plt.annotate('C=%d'%dic_rutas[(i,j)], ((direccion_x[n.index(i)]+direccion_x[n.index(j)])/2, (direccion_y[n.index(i)]+direccion_y[n.index(j)])/2))
plt.xlabel("Eje X")
plt.ylabel("Eje Y")
plt.title("Grafo completo, arcos entre todo par de nodos donde existe ruta| VRP")


plt.show()"""

Fuente: https://nbviewer.jupyter.org/github/CplexPython/Cplex-Python/blob/master/VRP/CVRP%20%7C%20Cplex%20%26%20Python%20.ipynb

# Utilizando Heurísticas para resolver VRP

In [ ]:
"""# Plot the shortest path
fig, ax = ox.plot_graph_route(G, route)"""